In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset

import numpy as np
import evaluate

In [2]:
# Load model directly
model_name = "boltuix/bert-lite"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load dataset
dataset_name = 'ucberkeley-dlab/measuring-hate-speech'
dataset = load_dataset(dataset_name, split='train')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at boltuix/bert-lite and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Check the shape of the dataset
dataset.shape

(135556, 131)

In [4]:
# Create a custom dataset from existing dataset. 
# Use 'text' as the input and encode 'hate_speech_score' as the label- if 'hate_speech_score' is > 0.5, it is considered hate speech, so encode it as 1, else 0.
# This classification is taken from the dataset's description:
# hate_speech_score - continuous hate speech measure, where higher = more hateful and lower = less hateful. > 0.5 is approximately hate speech, < -1 is counter or supportive speech, and -1 to +0.5 is neutral or ambiguous.
def encode_labels(example):
    example['label'] = 1 if example['hate_speech_score'] > 0.5 else 0
    return example

encoded_dataset = dataset.map(encode_labels)
encoded_dataset = encoded_dataset.remove_columns(['hate_speech_score'])

In [5]:
# Select only subset of the dataset for training and testing as there are too many examples in the dataset.
subset = encoded_dataset.train_test_split(test_size=0.2)['test']

# Split the dataset into train and test sets
train_test_split = subset.train_test_split(test_size=0.2, seed=42)

train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [6]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'])

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Remove the original text column and set the format for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['text'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(['text'])

tokenized_train_dataset.set_format('torch')
tokenized_test_dataset.set_format('torch')

# Rename the label column to 'labels' for compatibility with Trainer
tokenized_train_dataset = tokenized_train_dataset.rename_column('label', 'labels')
tokenized_test_dataset = tokenized_test_dataset.rename_column('label', 'labels')

Map:   0%|          | 0/21689 [00:00<?, ? examples/s]

Map:   0%|          | 0/5423 [00:00<?, ? examples/s]

In [7]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./fine-tuned-hatebert',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=False,
    fp16=False,
)

In [8]:
# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    
    return {
        'accuracy': acc['accuracy'],
        'f1': f1['f1'],
    }

In [9]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [10]:
# Train the model
trainer.train()

  0%|          | 0/6780 [00:00<?, ?it/s]

{'loss': 0.4361, 'grad_norm': 12.445770263671875, 'learning_rate': 1.8525073746312686e-05, 'epoch': 0.37}
{'loss': 0.3268, 'grad_norm': 10.969100952148438, 'learning_rate': 1.705014749262537e-05, 'epoch': 0.74}


  0%|          | 0/339 [00:00<?, ?it/s]

{'eval_loss': 0.2892463803291321, 'eval_accuracy': 0.8720265535681357, 'eval_f1': 0.8721663008407808, 'eval_runtime': 8.1246, 'eval_samples_per_second': 667.481, 'eval_steps_per_second': 41.725, 'epoch': 1.0}
{'loss': 0.3099, 'grad_norm': 8.052376747131348, 'learning_rate': 1.5575221238938054e-05, 'epoch': 1.11}
{'loss': 0.283, 'grad_norm': 9.49429988861084, 'learning_rate': 1.4100294985250738e-05, 'epoch': 1.47}
{'loss': 0.2748, 'grad_norm': 7.871114253997803, 'learning_rate': 1.2625368731563424e-05, 'epoch': 1.84}


  0%|          | 0/339 [00:00<?, ?it/s]

...

  0%|          | 0/339 [00:00<?, ?it/s]

{'eval_loss': 0.2768039107322693, 'eval_accuracy': 0.8886225336529596, 'eval_f1': 0.8881422175098741, 'eval_runtime': 7.3096, 'eval_samples_per_second': 741.898, 'eval_steps_per_second': 46.377, 'epoch': 4.0}
{'loss': 0.2238, 'grad_norm': 9.897957801818848, 'learning_rate': 3.775811209439528e-06, 'epoch': 4.06}
{'loss': 0.2005, 'grad_norm': 9.679115295410156, 'learning_rate': 2.3008849557522127e-06, 'epoch': 4.42}
{'loss': 0.2143, 'grad_norm': 6.320766925811768, 'learning_rate': 8.259587020648968e-07, 'epoch': 4.79}


  0%|          | 0/339 [00:00<?, ?it/s]

{'eval_loss': 0.28631293773651123, 'eval_accuracy': 0.8864097363083164, 'eval_f1': 0.8861232012918668, 'eval_runtime': 7.4008, 'eval_samples_per_second': 732.761, 'eval_steps_per_second': 45.806, 'epoch': 5.0}
{'train_runtime': 462.2818, 'train_samples_per_second': 234.586, 'train_steps_per_second': 14.666, 'train_loss': 0.2626581467710062, 'epoch': 5.0}


TrainOutput(global_step=6780, training_loss=0.2626581467710062, metrics={'train_runtime': 462.2818, 'train_samples_per_second': 234.586, 'train_steps_per_second': 14.666, 'total_flos': 210033065952180.0, 'train_loss': 0.2626581467710062, 'epoch': 5.0})

In [ ]:
# Save the model
save_directory = "../data/bert-lite/"
# trainer.save_model(save_directory, safe_serialization=False)
model.save_pretrained(save_directory, safe_serialization=False)
tokenizer.save_pretrained(save_directory)

('../data/bert-lite/tokenizer_config.json',
 '../data/bert-lite/special_tokens_map.json',
 '../data/bert-lite/vocab.txt',
 '../data/bert-lite/added_tokens.json',
 '../data/bert-lite/tokenizer.json')